In [1]:
!pip install qiskit


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install qiskit_aer


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from qiskit import QuantumCircuit , ClassicalRegister,QuantumRegister
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel , pauli_error
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

In [8]:
noise_model = NoiseModel()
bit_flip = pauli_error([('X', 0.1), ('I', 0.9)])
noise_model.add_all_qubit_quantum_error(bit_flip, ['id', 'x'])

In [9]:
def create_bit_flip_code():
    qr = QuantumRegister(3, 'q')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(qr, cr)
    qc.x(0) 
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.barrier()
    qc.id(0)
    qc.id(1)
    qc.id(2)
    qc.barrier()
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.ccx(1, 2, 0)
    qc.barrier()
    qc.measure(0, 0)
    return qc

In [10]:
qc = create_bit_flip_code()
print("Circuit:")
print(qc.draw())

Circuit:
     ┌───┐           ░ ┌───┐ ░           ┌───┐ ░ ┌─┐
q_0: ┤ X ├──■────■───░─┤ I ├─░───■────■──┤ X ├─░─┤M├
     └───┘┌─┴─┐  │   ░ ├───┤ ░ ┌─┴─┐  │  └─┬─┘ ░ └╥┘
q_1: ─────┤ X ├──┼───░─┤ I ├─░─┤ X ├──┼────■───░──╫─
          └───┘┌─┴─┐ ░ ├───┤ ░ └───┘┌─┴─┐  │   ░  ║ 
q_2: ──────────┤ X ├─░─┤ I ├─░──────┤ X ├──■───░──╫─
               └───┘ ░ └───┘ ░      └───┘      ░  ║ 
c: 1/═════════════════════════════════════════════╩═
                                                  0 


In [11]:
simulator = AerSimulator(noise_model=noise_model)
result_noisy = simulator.run(qc, shots=1000).result()
counts = result_noisy.get_counts()

In [12]:
print(f"\nResults with error correction:")
print(f"Correct (|1⟩): {counts.get('1', 0)/10:.1f}%")
print(f"Error (|0⟩): {counts.get('0', 0)/10:.1f}%")


Results with error correction:
Correct (|1⟩): 88.2%
Error (|0⟩): 11.8%


In [13]:
qc_no_ec = QuantumCircuit(1, 1)
qc_no_ec.x(0)
qc_no_ec.id(0)
qc_no_ec.measure(0, 0)
result_no_ec = simulator.run(qc_no_ec, shots=1000).result()
counts_no_ec = result_no_ec.get_counts()

In [14]:
print(f"\nResults WITHOUT error correction:")
print(f"Correct (|1⟩): {counts_no_ec.get('1', 0)/10:.1f}%")
print(f"Error (|0⟩): {counts_no_ec.get('0', 0)/10:.1f}%")


Results WITHOUT error correction:
Correct (|1⟩): 80.7%
Error (|0⟩): 19.3%
